## Muestreo sistemático
    - Es la forma de seleccionar una muestra donde únicamente el primer elemento es seleccionado aleatoriamente, mientras que el resto se toman sistemáticamente con intervalos constantes cada k elementos en la población.
        k = N/n (k entero)
    
    VENTAJAS:
    - Es más fácil hacer un muestreo, ya que el m.a.s. está sujeto a aleatoriedad, mientras que el sistemático puede aplicarse en campo sin incurrir en sesgos importantes.
    - La muestra se reparte sistemáticamente en toda la población, por lo que es más factible reproducir una muestra "representativa"
        
    DESVENTAJAS
    - Un mal arreglo en el marco puede producir muestras ineficientes
    - NO se pueden calcular estimadores de la varianza a partir de una sola muestra sistemática

### Conclusiones
    - Se podría decir que se escoge un elemento dentro de k estratos

    - El muestreo sistemático sí disminuye la varianza de los estimadores
    
    - Se debe cuidar que el orden de la población no sea periódico, ya que si coincide con el periodo de muestreo, se podría perder información o sobrestimar la varianza de los parámetros
    
    - Las estimaciones con múltiples muestreos sistemáticos sigue las reglas y fórmulas del muestreo en conglomerados. Cuando existe orden en la población, el estimador será más preciso que el m.a.s.
    
    - ***RO***
    
    - Cuando N no sea divisible exactamente entre n elementos, el tamaño de muestra variará entre k y k-1, dependiendo la semilla de arranque
    
    - El muestreo sistemático es eficiente cuando los elementos dentro de cada conglomerado son heterogenes y es ineficiente cuando son homogeneos

### 1] Definir una población
    - Se define a partir de una distribución normal

In [198]:
import random
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.stats import t

# Settngs
N = 300
n = 15
confidence = 0.95

# Population data frame
df_population = pd.DataFrame()
df_population["value"] = norm.rvs(loc=100,scale=25,size=N)

# Population parameters
mean = df_population["value"].mean(0)
total = df_population["value"].sum(0)

# Output
print("="*100)
print("PARÁMETROS POBLACIONALES\n")
print("Media = ", round(mean,1))
print("Total = ", round(total,1))
print("="*100)

PARÁMETROS POBLACIONALES

Media =  101.3
Total =  30386.8


### 2] Estimación con m.a.s.
    - Para fines comparativos

    - Cuando las unidades en el marco están en un orden aleatorio, el muestreo sistemático asemeja al m.a.s.

In [199]:
# Select a sample
df_sample = df_population.sample(n=n, random_state=314159)

# Point estimates
mean = df_sample["value"].mean(0)
total = N * df_sample["value"].mean(0)
var = df_sample["value"].var(0,ddof=1)

# Confidence intervals
mean_std = ( (1-n/N) * var/n )**0.5
mean_interval = mean + mean_std*t(df=n-1).ppf((0.5-confidence/2, 0.5+confidence/2))

# Output
print("="*100)
print("ESTIMACIÓN CON M.A.S.")
print("EE(Media):",round(mean_std,1))
print("\nMedia:",round(mean,1))
print("Entre:",np.round(mean_interval,1))
print("="*100)

ESTIMACIÓN CON M.A.S.
EE(Media): 4.9

Media: 99.0
Entre: [ 88.6 109.4]


### 3] Estimación con una muestra sistemática
    - Para este caso las expresiones utilizadas corresponden a las del m.a.s.
    
    - Cuando la muestra se encuentra ordenada, se esperan mejores reultados que el m.a.s.
    
    - Se selecciona un arranque i aleatorio tal que 1<=i<=k (se selecciona un aleatorio entre los primeros k elementos). Por lo tanto la unidad i U_[i] es el arranque. A continuación, la unidad que se encuentre a k unidades de distancia es seleccionada U_[i+k], posteriormente U_[i+2k]... hasta U_[i-(n-1)k]
    
    - Otra forma de seleccionar la muestra es arrancar en k/2, con lo cuál se toma la mitad del conglomerado de k elementos y, teóricamente, se mejora la precisión de los estimadores, mientras que otros investigadores sugieren que realizar esto haría que los estimadores se comporten errántemente.
    
    - Para una muestra sistemática se tiene que:
        y_mean = sum(y_i)/n

In [200]:
# Sort increasingly
df_population.sort_values(by=["value"], inplace=True)

# Define steps in choosing units
k = int(N/n)

# Randomly select a start unit
start = int(np.random.randint(low=0, high=k, size=1))

# Mask indicating element to be included in sample
mask = [0]*k
mask[start] = 1
mask = np.tile(mask, n).astype(bool)

# Select sample
df_sample = df_population[mask]



# Point estimates
mean = df_sample["value"].mean(0)
total = N * df_sample["value"].mean(0)
var = df_sample["value"].var(0,ddof=1)

# Confidence intervals
mean_std = ( (1-n/N) * var/n )**0.5
mean_interval = mean + mean_std*t(df=n-1).ppf((0.5-confidence/2, 0.5+confidence/2))

# Output
print("="*100)
print("ESTIMACIÓN CON MUESTREO SISTEMÁTICO")
print("EE(Media):",round(mean_std,1))
print("\nMedia:",round(mean,1))
print("Entre:",np.round(mean_interval,1))
print("="*100)

ESTIMACIÓN CON MUESTREO SISTEMÁTICO
EE(Media): 6.2

Media: 103.7
Entre: [ 90.5 117. ]


### 4] Seleccionar múltiples muestras sistemáticas
    - Es equivalente seleccionar diferentes arranques o tomar todas las muestras posibles y seleccionarlas

    - Para mejorar los estimadores es necesario tomar mútiples muestras de tamaño n, no dividir n entre el número de muestras a tomar

In [201]:
# Sample size is divided by two to work with three clusters
k = 2

# Define all possible systematic samples
df_sample = df_population.copy()
df_sample["sample"] = np.tile([i for i in range(int(N/n))],n)
    
# Randomly select k samples
mask = random.sample([i for i in range(int(N/n))], k)
df_sample = df_sample[df_sample["sample"].isin(mask)].copy()

# Statistics of systematic samples
sample = df_sample["sample"]
sample_mean = [df_sample["value"].mean(0)] + [df_sample["value"][sample==i].mean(0) for i in mask]
sample_var = [df_sample["value"].var(0, ddof=1)] + [df_sample["value"][sample==i].var(0,ddof=1) for i in mask]

### 5] Estimación con múltiples muestras sistemáticas

    - En realidad, lo que se hace es dividir la población en k muestras (conglomerados). Existen k muestras posibles, el mismo número que los saltos en los intervalos.
    
    - La media poblacional se estima como:
    
        mean = 1/k * sum(mean_j)
            
            donde:
            k: Número de muestras sistemáticas utilizadas
            mean_j: Media dentro de cada la muestra sistemática j
            

    - La varianza del estimador de la media se calcula como.
        
        V(y_mean) = (1-1/N)*S^2 - (1-1/n)*S_w^2
        S_w^2 = 1/k * sum[j in k](S_j)
    
        donde:
            S^2: Varianza observada al unir los elementos de todas las muestras sistemáticas
            k: Número de muestras sistemáticas seleccionadas
            n: Tamaño de muestra dentro de cada muestra sistemática
            S_w^2: Varianza (within) dentro conglomerados
            S_j: Varianza observada dentro del conglomerado j
            Nota: Observe que S_w^2 es el promedio de las varianzas observadas dentro de los conglomerados
            Nota: (1-1/N) es casi equivalente a k*(n-1)/N
            
    - Para que V(y_mean) de un muestreo sistemático sea mejor estimador que el m.a.s. se necesita que S < S_w, es decir, que la varianza promedio de los conglomerados sea mayor que una muestra unificada.

In [202]:
# Compute mean
mean = np.array([df_sample["value"][sample==i].mean(0) for i in mask]).mean(0)

# Compute variance of estimator of mean
var_w = np.array(sample_var[1:]).mean(0)
mean_var = (1-1/N)*sample_var[0] - (N/n)*(n-1)/N*var_w
mean_std = mean_var**0.5
mean_interval = mean + mean_std*t(df=n-1).ppf((0.5-confidence/2, 0.5+confidence/2))

# Output
print("="*100)
print("ESTIMACIÓN CON MÚLTIPLES MUESTRAS SISTEMÁTICAS")
print("EE(Media):",round(mean_std,1))
print("Media:",round(mean,1))
print("Entre:",np.round(mean_interval,1))
print("\nCONSIDERANDO UNA SOLA MUESTRA:\nEE(Media)[m.a.s]:",
      round((df_sample["value"].var(0) / n*k * (1-n*k/N))**0.5,1))
print("EE(Media)[single systematic]:",
     round((df_sample["value"][sample==mask[0]].var(0) / n * (1-n/N))**0.5,1))
print("="*100)

ESTIMACIÓN CON MÚLTIPLES MUESTRAS SISTEMÁTICAS
EE(Media): 4.5
Media: 101.3
Entre: [ 91.5 111. ]

CONSIDERANDO UNA SOLA MUESTRA:
EE(Media)[m.a.s]: 8.2
EE(Media)[single systematic]: 6.1


### 6] Otra forma de V(mean)
    - Alternativamente, existe otra expresión de la varianza de la media.
        
        V(mean) = S_st^2/n * (1-1/N) * (1+ro*(n-1))
        S_wst^2 = 1/(n*k-n) * sum[j in n](sum[i in k]( (y_ij-mean_j)^2 ))
        ro = 2/(n*(n-1)*(k-1)*S_wst^2) * sum[r in k](sum[i in n][j!=i in n] ((y_i-mean_st_i)*(y_j-mean_st_j)))
        
            donde:
            S_wst^2: Varianza de las unidades dentro de cada estrato
            ro: Coeficiente de correlación intraclase
            mean_st_i: Media del estrato del elemento i (a través de las k muestras sistemáticas)
            
            Nota: Debido a que se forman k estratos en la población, al parear las unidades de cada muestra
            sistemática, las primeras de ellas se extraen del primer estrato (1 a k unidades de la población),
            las segundas unidades, del segundo estrato (k+1 a 2k) y sucesivamente

            Nota: Observe que para S_st^2 se itera primero por elemento y por columna, contrario a lo usual
            que es que para cada columna se iteren todos los elementos
            
            Nota: Observe que ro se calcula sumando para cada muestra sistemática el producto de la diferencia 
            de la media de cada estrato para el cruce de todos los elementos dentro de cada muestra sistemática

In [203]:
# Compute statistics for stratas
mean = [df_sample["value"][i*k:k*(i+1)].mean(0) for i in range(n)]
var_wst = 1/(n*k-n) * sum(sum( (df_sample["value"].iloc[j*k+i] - mean[j])**2 for i in range(k)) for j in range(n))

# Compute ro
accum = 0
for i in range(k):
    values = np.array(df_sample["value"][sample==mask[0]])
    for j in range(n):
        for u in range(j+1,n):
            accum += (values[j]-mean[j]) * (values[u]-mean[u])
ro = 2 / (n*(n-1)*(k-1)*var_wst) * accum

# Compute estimators
mean = np.array([df_sample["value"][sample==i].mean(0) for i in mask]).mean(0)
mean_var = var_wst/n * (1-n/N) * (1 + (n-1)*ro)
mean_std = mean_var**0.5
mean_interval = mean + mean_std*t(df=n-1).ppf((0.5-confidence/2, 0.5+confidence/2))

# Output
print("="*100)
print("ESTIMACIÓN CON MÚLTIPLES MUESTRAS SISTEMÁTICAS")
print("EE(Media):",round(mean_std,1))
print("Media:",round(mean,1))
print("Entre:",np.round(mean_interval,1))
print("="*100)


ESTIMACIÓN CON MÚLTIPLES MUESTRAS SISTEMÁTICAS
EE(Media): 2.7
Media: 101.3
Entre: [ 95.4 107.1]
